In [1]:
import os
import sqlite3
import pandas as pd
from datetime import datetime
from fuzzywuzzy import process

In [2]:
conn = sqlite3.connect("../data/db/NBA-L2M.db")
c = conn.cursor()

In [34]:
reports = pd.read_sql("SELECT * FROM reports", conn, parse_dates=['date'])
reports.replace({'away':{'PHO': 'PHX'}})
reports.date = pd.to_datetime(reports.date.dt.date)
reports['idx'] = reports.away + reports.home + reports.date.dt.strftime("%Y-%m-%d")
print reports.dtypes, '\r'
reports.head(2)

report                object
away                  object
home                  object
date          datetime64[ns]
season                object
away_score            object
home_score            object
winner                object
idx                   object
dtype: object 


,report,away,home,date,season,away_score,home_score,winner,idx
0,L2M-DEN-MIN-11-03-16,DEN,MIN,2016-11-02,2016-17 NBA Regular Season,NaN,NaN,NaN,DENMIN2016-11-02
1,L2M-SAC-CHI-01-21-17,SAC,CHI,2017-01-20,2016-17 NBA Regular Season,NaN,NaN,NaN,SACCHI2017-01-20


In [35]:
calls = pd.read_sql("SELECT * FROM calls", conn)
print calls.dtypes, '\r'
calls.head(2)

report                object
period                object
time                  object
call                  object
committing            object
disadvantaged         object
decision              object
committing_team       object
disadvantaged_team    object
dtype: object 


,report,period,time,call,committing,disadvantaged,decision,committing_team,disadvantaged_team
0,L2M-DEN-MIN-11-03-16,Q4,,Foul: Shooting,Karl-Anthony Towns,Emmanuel Mudiay,CC,MIN,DEN
1,L2M-DEN-MIN-11-03-16,Q4,,Foul: Offensive,Karl-Anthony Towns,Danilo Gallinari,CNC,MIN,DEN


In [36]:
tms = {'Atlanta Hawks': 'ATL', 'Boston Celtics': 'BOS',
       'Brooklyn Nets': 'BKN', 'Charlotte Hornets': 'CHA',
       'Chicago Bulls': 'CHI', 'Cleveland Cavaliers': 'CLE',
       'Dallas Mavericks': 'DAL', 'Denver Nuggets': 'DEN',
       'Detroit Pistons': 'DET', 'Golden State Warriors': 'GSW',
       'Houston Rockets': 'HOU', 'Indiana Pacers': 'IND',
       'Los Angeles Clippers': 'LAC', 'Los Angeles Lakers': 'LAL',
       'Memphis Grizzlies': 'MEM', 'Miami Heat': 'MIA',
       'Milwaukee Bucks': 'MIL', 'Minnesota Timberwolves': 'MIN',
       'New Orleans Pelicans': 'NOP', 'New York Knicks': 'NYK',
       'Oklahoma City Thunder': 'OKC', 'Orlando Magic': 'ORL',
       'Philadelphia 76ers': 'PHI', 'Phoenix Suns': 'PHX', 
       'Portland Trail Blazers': 'POR', 'Sacramento Kings': 'SAC',
       'San Antonio Spurs': 'SAS', 'Toronto Raptors': 'TOR',
       'Utah Jazz': 'UTA', 'Washington Wizards': 'WAS'}

In [38]:
gms = pd.read_sql("SELECT * FROM 'scheduled-gms'", conn)
gms = gms[gms.Date != 'Date']
gms.Date = pd.to_datetime(gms["Date"], format="%a %b %d %Y")
gms.columns = ["Date", "Start", "Visitor", "V_PTS", "Home", "H_PTS", "Boxscore", "OT", "Notes"]
gms.replace({'Visitor': tms, 'Home': tms}, inplace=True)
gms['idx'] = gms.Visitor + gms.Home + gms.Date.dt.strftime("%Y-%m-%d")
print gms.dtypes, '\r'
gms.head(2)

Date        datetime64[ns]
Start               object
Visitor             object
V_PTS               object
Home                object
H_PTS               object
Boxscore            object
OT                  object
Notes               object
idx                 object
dtype: object 


,Date,Start,Visitor,V_PTS,Home,H_PTS,Boxscore,OT,Notes,idx
0,2017-06-01,9:00 pm,CLE,91,GSW,113,Box Score,None,None,CLEGSW2017-06-01
1,2017-06-04,8:00 pm,CLE,113,GSW,132,Box Score,None,None,CLEGSW2017-06-04


In [42]:
pd.merge(reports, gms)

,report,away,home,date,season,away_score,home_score,winner,idx,Date,Start,Visitor,V_PTS,Home,H_PTS,Boxscore,OT,Notes


In [20]:
print len(calls)
print len(reports)
reports.iloc[0].date

15625
963


Timestamp('2016-11-02 00:00:00')

In [21]:
gms.iloc[0].Date

Timestamp('2017-06-01 00:00:00')

In [47]:
gms.sort_values("idx").head(20)

,Date,Start,Visitor,V_PTS,Home,H_PTS,Boxscore,OT,Notes,idx
3484,2014-12-05,7:30 pm,ATL,98,BKN,75,Box Score,None,None,ATLBKN2014-12-05
2717,2015-04-08,7:30 pm,ATL,114,BKN,111,Box Score,None,None,ATLBKN2015-04-08
2810,2015-04-25,3:00 pm,ATL,83,BKN,91,Box Score,None,None,ATLBKN2015-04-25
2818,2015-04-27,7:00 pm,ATL,115,BKN,120,Box Score,OT,None,ATLBKN2015-04-27
2632,2015-05-01,8:00 pm,ATL,111,BKN,87,Box Score,None,None,ATLBKN2015-05-01
2484,2015-11-17,7:30 pm,ATL,88,BKN,90,Box Score,None,None,ATLBKN2015-11-17
644,2017-01-10,7:30 pm,ATL,117,BKN,97,Box Score,None,None,ATLBKN2017-01-10
39,2017-04-02,6:00 pm,ATL,82,BKN,91,Box Score,None,None,ATLBKN2017-04-02
3316,2015-01-14,8:00 pm,ATL,105,BOS,91,Box Score,None,None,ATLBOS2015-01-14
3135,2015-02-11,7:30 pm,ATL,88,BOS,89,Box Score,None,None,ATLBOS2015-02-11


In [48]:
reports.sort_values("idx").head()

,report,away,home,date,season,away_score,home_score,winner,idx
620,L2M-ATL-BKN-04-08-15,ATL,BKN,2015-04-07,2014-15 NBA Regular Season,NaN,NaN,NaN,ATLBKN2015-04-07
2,L2M-ATL-BKN-04-27-15,ATL,BKN,2015-04-26,2014-15 NBA Playoffs,NaN,NaN,NaN,ATLBKN2015-04-26
421,L2M-ATL-BKN-11-17-15,ATL,BKN,2015-11-16,2015-16 NBA Regular Season,NaN,NaN,NaN,ATLBKN2015-11-16
368,L2M-ATL-BOS-12-18-15,ATL,BOS,2015-12-17,2015-16 NBA Regular Season,NaN,NaN,NaN,ATLBOS2015-12-17
196,L2M-ATL-BOS-04-22-16,ATL,BOS,2016-04-21,2015-16 NBA Playoffs,NaN,NaN,NaN,ATLBOS2016-04-21
